# 프로그램_1에서 수집되고 있는 실시간 raw데이터를 10초 단위로 정리하고, 이를 기준으로 트레이딩 결정까지 내리는 프로그램

In [1]:
import pyupbit
import uuid
import json
import asyncio
import websockets
import traceback
import nest_asyncio
import pandas as pd
import datetime
import time
nest_asyncio.apply()

In [2]:
####### 오전 8시부터 밤 12시 이전에만 트레이딩하도록 수정해야.

async def data_manipulation(lock):
    
    my_data = pd.DataFrame(columns=['datetime_10s', 'beginning_orderbook_midprice','beginning_price', 'highest_price',
                                       'lowest_price', 'ending_price','ending_orderbook_midprice',
                                       'trading_volume', 'volume_power','trade_volume_bid', 'trade_volume_ask',
                                       'ending_orderbook_total_ask_size', 'ending_orderbook_total_bid_size', 
                                        'ending_orderbook_ap_0','ending_orderbook_as_0',
                                        'ending_orderbook_ap_1','ending_orderbook_as_1',
                                       'ending_orderbook_ap_2','ending_orderbook_as_2',
                                       'ending_orderbook_ap_3','ending_orderbook_as_3',
                                       'ending_orderbook_ap_4','ending_orderbook_as_4',
                                       'ending_orderbook_ap_5','ending_orderbook_as_5',
                                       'ending_orderbook_ap_6','ending_orderbook_as_6',
                                       'ending_orderbook_ap_7','ending_orderbook_as_7',
                                       'ending_orderbook_ap_8','ending_orderbook_as_8',
                                       'ending_orderbook_ap_9','ending_orderbook_as_9',
                                       'ending_orderbook_ap_10','ending_orderbook_as_10',
                                       'ending_orderbook_ap_11','ending_orderbook_as_11',
                                       'ending_orderbook_ap_12','ending_orderbook_as_12',
                                       'ending_orderbook_ap_13','ending_orderbook_as_13',
                                       'ending_orderbook_ap_14','ending_orderbook_as_14',
                                        'ending_orderbook_bp_0','ending_orderbook_bs_0',
                                        'ending_orderbook_bp_1','ending_orderbook_bs_1',
                                       'ending_orderbook_bp_2','ending_orderbook_bs_2',
                                       'ending_orderbook_bp_3','ending_orderbook_bs_3',
                                       'ending_orderbook_bp_4','ending_orderbook_bs_4',
                                       'ending_orderbook_bp_5','ending_orderbook_bs_5',
                                       'ending_orderbook_bp_6','ending_orderbook_bs_6',
                                       'ending_orderbook_bp_7','ending_orderbook_bs_7',
                                       'ending_orderbook_bp_8','ending_orderbook_bs_8',
                                       'ending_orderbook_bp_9','ending_orderbook_bs_9',
                                       'ending_orderbook_bp_10','ending_orderbook_bs_10',
                                       'ending_orderbook_bp_11','ending_orderbook_bs_11',
                                       'ending_orderbook_bp_12','ending_orderbook_bs_12',
                                       'ending_orderbook_bp_13','ending_orderbook_bs_13',
                                       'ending_orderbook_bp_14','ending_orderbook_bs_14'])

    my_data['datetime_10s'] = pd.to_datetime(my_data['datetime_10s'],errors='raise')
    
    while True:
        # Get the current time
        
        now = datetime.datetime.now()
#         print("new iteration with now:", now)
        
        #  10초 기준 중간 즈음에 있을 때에는 프로그램을 잠시 쉬어준다 (과도한 무한루프를 막기 위함)
        if ((1< now.second % 10)  & (now.second % 10 < 8)):
            time.sleep(1)
            continue
        
        # Check if the seconds are a unit second (0/10/20/30/40/50s)
        if now.second % 10 == 0:
            try:
                async with lock:
                    print("CHECKER#0")
                    df2 = pd.read_csv('temp3.csv')
                    print("CHECKER#0_1")

                    df2 = df2.copy()
#                     print("now in the lock & df2:", df2)
                    df2['mytime'] = pd.to_datetime(df2['mytime'],errors='raise')
                    print("CHECKER#0_2")
                    df2.sort_values(by = 'mytime')

#                     print("df['mytime'].iloc[-1].second", df['mytime'].iloc[-1].second)
#                     print("df['mytime'].iloc[-1].minute", df['mytime'].iloc[-1].minute)

                    # if ((now.second -1 == df['mytime'].iloc[-1].second) & (now.minute == df['mytime'].iloc[-1].minute)) | ( (now.second ==0 &  df['mytime'].iloc[-1].second == 59) & (now.minute -1 == df['mytime'].iloc[-1].minute))
                    # | ( (now.second ==0 &  df['mytime'].iloc[-1].second == 59) & (now.minute ==0 & df['mytime'].iloc[-1].minute==59) & )

                    if ((now.second -1 == df2['mytime'].iloc[-1].second) | ((now.second == 0) & (df2['mytime'].iloc[-1].second==59)) 
                        | (now.second == df2['mytime'].iloc[-1].second)):
                        
                        # 이미 데이터가 있을 수도 있다. 그러면 loop의 다음으로 continue
                        if (my_data.shape[0]>0):
                            delta = now - my_data['datetime_10s'].iloc[-1]
                            if delta.seconds < 11 :
                                continue
                
                        print("CHECKER#1")

                        # 만약 이미 10초 구간에 발생한 tick data가 들어와 있다면 drop한다.
                        while now.second == df2['mytime'].iloc[-1].second:
                            # print("CHECKER#2")
                            # print("now.second:", now.second)
                            # print("df2['mytime'].iloc[-1].second:",df2['mytime'].iloc[-1].second)
#                             print(df2.iloc[-1])
                            df2 = df2.drop([df2.shape[0]-1])
                    
                        # make a 10s line.
                        # if one is existing, then pass.
                        
                        print("inside:" )
                        print("now:", now)

                        
                        # print("df2['mytime'].iloc[-1].second:", df2['mytime'].iloc[-1].second)
#                         print( "df['datetime'].iloc[-1].second:", df['datetime'].iloc[-2].second)

                        ## 10초 동안 다음의 변수값을 도출 할 예정 (일단 모두 0으로 세팅)
                        datetime_10s=0 
                        beginning_orderbook_midprice = 0 ## ask price 0과 bid price 0의 평균 
                        beginning_price = 0 ## 10초 시작하고 시가
                        highest_price = 0 ## 고가
                        lowest_price = 0 ## 저가
                        ending_price = 0 ## 10초 끝나기 전 시가
                        ending_orderbook_midprice = 0

                        trading_volume = 0
                        volume_power = 0  ## 체결강도 (BID인 volume / ASK인 volume) ## bid(매수)가 더 많은지 ask(매도)가 더 많은지 비율로 

                        trade_volume_bid = 0 ## voluem power 계산용
                        trade_volume_ask = 0 ## voluem power 계산용

                        ending_orderbook_total_ask_size = 0
                        ending_orderbook_total_bid_size = 0
                        ending_orderbook_ap_0 = 0
                        ending_orderbook_as_0 = 0
                        ending_orderbook_ap_1 = 0
                        ending_orderbook_as_1 = 0
                        ending_orderbook_ap_2 = 0
                        ending_orderbook_as_2 = 0
                        ending_orderbook_ap_3 = 0
                        ending_orderbook_as_3 = 0
                        ending_orderbook_ap_4 = 0
                        ending_orderbook_as_4 = 0
                        ending_orderbook_ap_5 = 0
                        ending_orderbook_as_5 = 0
                        ending_orderbook_ap_6 = 0
                        ending_orderbook_as_6 = 0
                        ending_orderbook_ap_7 = 0
                        ending_orderbook_as_7 = 0
                        ending_orderbook_ap_8 = 0
                        ending_orderbook_as_8 = 0
                        ending_orderbook_ap_9 = 0
                        ending_orderbook_as_9 = 0
                        ending_orderbook_ap_10 = 0
                        ending_orderbook_as_10 = 0
                        ending_orderbook_ap_11 = 0
                        ending_orderbook_as_11 = 0
                        ending_orderbook_ap_12 = 0
                        ending_orderbook_as_12 = 0
                        ending_orderbook_ap_13 = 0
                        ending_orderbook_as_13 = 0
                        ending_orderbook_ap_14 = 0
                        ending_orderbook_as_14 = 0

                        ending_orderbook_bp_0 = 0
                        ending_orderbook_bs_0 = 0
                        ending_orderbook_bp_1 = 0
                        ending_orderbook_bs_1 = 0
                        ending_orderbook_bp_2 = 0
                        ending_orderbook_bs_2 = 0
                        ending_orderbook_bp_3 = 0
                        ending_orderbook_bs_3 = 0
                        ending_orderbook_bp_4 = 0
                        ending_orderbook_bs_4 = 0
                        ending_orderbook_bp_5 = 0
                        ending_orderbook_bs_5 = 0
                        ending_orderbook_bp_6 = 0
                        ending_orderbook_bs_6 = 0
                        ending_orderbook_bp_7 = 0
                        ending_orderbook_bs_7 = 0
                        ending_orderbook_bp_8 = 0
                        ending_orderbook_bs_8 = 0
                        ending_orderbook_bp_9 = 0
                        ending_orderbook_bs_9 = 0
                        ending_orderbook_bp_10 = 0
                        ending_orderbook_bs_10 = 0
                        ending_orderbook_bp_11 = 0
                        ending_orderbook_bs_11 = 0
                        ending_orderbook_bp_12 = 0
                        ending_orderbook_bs_12 = 0
                        ending_orderbook_bp_13 = 0
                        ending_orderbook_bs_13 = 0
                        ending_orderbook_bp_14 = 0
                        ending_orderbook_bs_14 = 0

                        ## 실제로 10초를 한 줄로 만드는 작업 시작
                        j = 0

                        # df2['datetime'].iloc[i].second//10
                        # while  ( (df2[-1-j].get('datetime').second//10 == df2[-2].get('datetime').second//10)  &   df2[-1-j].get('datetime').second%%10==9 ) | df2[-1-j].get('datetime').second%%10!=9 :
#                         while ((df['datetime'].iloc[-1-j].second//10 == df['datetime'].iloc[-1].second//10) & df['datetime'].iloc[-1-j].second%10 == 9) | df['datetime'].iloc[-1-j].second%10 != 9:
                        while (df2['mytime'].iloc[-1-j].second//10 == df2['mytime'].iloc[-1].second//10):
                            
            
                            print("CHECKER#3")
                            # print("df2['type'].iloc[-1-j]:", df2['type'].iloc[-1-j])
            
                            #if (df2[-1-j].get('type_websocket') == 'ticker') & (ending_price==0):
                            if (df2['type'].iloc[-1-j] == 'ticker') & (ending_price==0):
                                print("CHECKER#4")

                                # ending_price = df2[-1-j].get('trade_price')
                                ending_price = df2['trade_price'].iloc[-1-j]

#                                 elif (df2[-1-j].get('type_websocket') == 'orderbook')  & (ending_orderbook_midprice==0):
                            elif (df2['type'].iloc[-1-j] == 'orderbook') & (ending_orderbook_midprice==0):
                                # print("CHECKER#5")

                                ending_orderbook_midprice = (df2['orderbook_ap_0'].iloc[-1-j] +  df2['orderbook_bp_0'].iloc[-1-j])/2
                                ending_orderbook_total_ask_size = df2['total_ask_size'].iloc[-1-j]
                                ending_orderbook_total_bid_size = df2['total_bid_size'].iloc[-1-j]
                                ending_orderbook_ap_0 = df2['orderbook_ap_0'].iloc[-1-j]/ending_orderbook_midprice  ## 나누는 이유: orderbook 평균가 기준 얼마나 높은지 비율로 따지기 위해 
                                ending_orderbook_as_0 = df2['orderbook_as_0'].iloc[-1-j]
                                ending_orderbook_ap_1 = df2['orderbook_ap_1'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_1 = df2['orderbook_as_1'].iloc[-1-j]
                                ending_orderbook_ap_2 = df2['orderbook_ap_2'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_2 = df2['orderbook_as_2'].iloc[-1-j]
                                ending_orderbook_ap_3 = df2['orderbook_ap_3'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_3 = df2['orderbook_as_3'].iloc[-1-j]
                                ending_orderbook_ap_4 = df2['orderbook_ap_4'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_4 = df2['orderbook_as_4'].iloc[-1-j]
                                ending_orderbook_ap_5 = df2['orderbook_ap_5'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_5 = df2['orderbook_as_5'].iloc[-1-j]
                                ending_orderbook_ap_6 = df2['orderbook_ap_6'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_6 = df2['orderbook_as_6'].iloc[-1-j]
                                ending_orderbook_ap_7 = df2['orderbook_ap_7'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_7 = df2['orderbook_as_7'].iloc[-1-j]
                                ending_orderbook_ap_8 = df2['orderbook_ap_8'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_8 = df2['orderbook_as_8'].iloc[-1-j]
                                ending_orderbook_ap_9 = df2['orderbook_ap_9'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_9 = df2['orderbook_as_9'].iloc[-1-j]
                                ending_orderbook_ap_10 = df2['orderbook_ap_10'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_10 = df2['orderbook_as_10'].iloc[-1-j]
                                ending_orderbook_ap_11 = df2['orderbook_ap_11'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_11 = df2['orderbook_as_11'].iloc[-1-j]
                                ending_orderbook_ap_12 = df2['orderbook_ap_12'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_12 = df2['orderbook_as_12'].iloc[-1-j]
                                ending_orderbook_ap_13 = df2['orderbook_ap_13'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_13 = df2['orderbook_as_13'].iloc[-1-j]
                                ending_orderbook_ap_14 = df2['orderbook_ap_14'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_as_14 = df2['orderbook_as_14'].iloc[-1-j]
                                ending_orderbook_bp_0 = df2['orderbook_bp_0'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_0 = df2['orderbook_bs_0'].iloc[-1-j]
                                ending_orderbook_bp_1 = df2['orderbook_bp_1'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_1 = df2['orderbook_bs_1'].iloc[-1-j]
                                ending_orderbook_bp_2 = df2['orderbook_bp_2'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_2 = df2['orderbook_bs_2'].iloc[-1-j]
                                ending_orderbook_bp_3 = df2['orderbook_bp_3'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_3 = df2['orderbook_bs_3'].iloc[-1-j]
                                ending_orderbook_bp_4 = df2['orderbook_bp_4'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_4 = df2['orderbook_bs_4'].iloc[-1-j]
                                ending_orderbook_bp_5 = df2['orderbook_bp_5'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_5 = df2['orderbook_bs_5'].iloc[-1-j]
                                ending_orderbook_bp_6 = df2['orderbook_bp_6'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_6 = df2['orderbook_bs_6'].iloc[-1-j]
                                ending_orderbook_bp_7 = df2['orderbook_bp_7'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_7 = df2['orderbook_bs_7'].iloc[-1-j]
                                ending_orderbook_bp_8 = df2['orderbook_bp_8'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_8 = df2['orderbook_bs_8'].iloc[-1-j]
                                ending_orderbook_bp_9 = df2['orderbook_bp_9'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_9 = df2['orderbook_bs_9'].iloc[-1-j]
                                ending_orderbook_bp_10 = df2['orderbook_bp_10'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_10 = df2['orderbook_bs_10'].iloc[-1-j]
                                ending_orderbook_bp_11 = df2['orderbook_bp_11'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_11 = df2['orderbook_bs_11'].iloc[-1-j]
                                ending_orderbook_bp_12 = df2['orderbook_bp_12'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_12 = df2['orderbook_bs_12'].iloc[-1-j]
                                ending_orderbook_bp_13 = df2['orderbook_bp_13'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_13 = df2['orderbook_bs_13'].iloc[-1-j]
                                ending_orderbook_bp_14 = df2['orderbook_bp_14'].iloc[-1-j]/ending_orderbook_midprice
                                ending_orderbook_bs_14 = df2['orderbook_bs_14'].iloc[-1-j]   

                            #if (df2[-1-j].get('type_websocket') == 'ticker'):
                            if df2['type'].iloc[-1-j] == 'ticker':
                                  ## 고가 수정 및 저가 수정
                                if highest_price <  df2['trade_price'].iloc[-1-j]: 
                                    highest_price = df2['trade_price'].iloc[-1-j]
                                if lowest_price == 0 or lowest_price > df2['trade_price'].iloc[-1-j]:
                                    lowest_price = df2['trade_price'].iloc[-1-j]

                                ## 거래량 더해주기
                                trading_volume += df2['trade_volume'].iloc[-1-j] 

                                # ask_bid => 매수/매도 구분 한 후에, 체결강도 계산용 BID/ASK 거래량 더해주기
                                if df2['ask_bid'].iloc[-1-j] =='BID': 
                                    trade_volume_bid += df2['trade_volume'].iloc[-1-j]
                                elif df2['ask_bid'].iloc[-1-j] =='ASK':
                                    trade_volume_ask += df2['trade_volume'].iloc[-1-j]

                            j+=1

                        datetime_10s = df2['mytime'].iloc[-1-j+1]

                        
                        print("CHECKER#6")
                        # print("df2['mytime'].iloc[-1-j+1].second:", df2['mytime'].iloc[-1-j+1].second)

                        
                        # 다시 거꾸로 '앞으로' 진행하면서 beginning 관련 변수값 얻기.
                        k=df2.shape[0]-1-j+1
                        while ((beginning_orderbook_midprice==0) | (beginning_price==0) ):
                            # if (df2[j+1+k].get('type_websocket') == 'ticker') & (beginning_price==0):
                            if (df2['type'].iloc[k] == 'ticker') & (beginning_price==0):
                                # beginning_price = df2[j+1+k].get('trade_price')
                                beginning_price = df2['trade_price'].iloc[k]
                            # elif (df2[j+1+k].get('type_websocket') == 'orderbook ') & (beginning_orderbook_midprice==0):
                            elif (df2['type'].iloc[k] == 'orderbook') & (beginning_orderbook_midprice==0):
                                beginning_orderbook_midprice = (df2['orderbook_ap_0'].iloc[k] +  df2['orderbook_bp_0'].iloc[k])/2
                            k+=1
                        
                        print("CHECKER#7")

                        volume_power = trade_volume_bid-trade_volume_ask

                        data_to_insert = pd.Series({'datetime_10s':datetime_10s, 'beginning_orderbook_midprice':beginning_orderbook_midprice,
                                                         'beginning_price':beginning_price, 'highest_price':highest_price,
                                                         'highest_price':highest_price, 'lowest_price':lowest_price,
                                                         'ending_price':ending_price, 'ending_orderbook_midprice':ending_orderbook_midprice,
                                                         'trading_volume':trading_volume, 'volume_power':volume_power,
                                                          'trade_volume_bid':trade_volume_bid, 'trade_volume_ask':trade_volume_ask,
                                                         'ending_orderbook_total_ask_size':ending_orderbook_total_ask_size, 'ending_orderbook_total_bid_size':ending_orderbook_total_bid_size,
                                                         'ending_orderbook_ap_0':ending_orderbook_ap_0, 'ending_orderbook_as_0':ending_orderbook_as_0,
                                                         'ending_orderbook_ap_1':ending_orderbook_ap_1, 'ending_orderbook_as_1':ending_orderbook_as_1,
                                                         'ending_orderbook_ap_2':ending_orderbook_ap_2, 'ending_orderbook_as_2':ending_orderbook_as_2,
                                                         'ending_orderbook_ap_3':ending_orderbook_ap_3, 'ending_orderbook_as_3':ending_orderbook_as_3,
                                                          'ending_orderbook_ap_4':ending_orderbook_ap_4, 'ending_orderbook_as_4':ending_orderbook_as_4,
                                                         'ending_orderbook_ap_5':ending_orderbook_ap_5, 'ending_orderbook_as_5':ending_orderbook_as_5,
                                                          'ending_orderbook_ap_6':ending_orderbook_ap_6, 'ending_orderbook_as_6':ending_orderbook_as_6,
                                                         'ending_orderbook_ap_7':ending_orderbook_ap_7, 'ending_orderbook_as_7':ending_orderbook_as_7,
                                                          'ending_orderbook_ap_8':ending_orderbook_ap_8, 'ending_orderbook_as_8':ending_orderbook_as_8,
                                                         'ending_orderbook_ap_9':ending_orderbook_ap_9, 'ending_orderbook_as_9':ending_orderbook_as_9,
                                                          'ending_orderbook_ap_10':ending_orderbook_ap_10, 'ending_orderbook_as_10':ending_orderbook_as_10,
                                                         'ending_orderbook_ap_11':ending_orderbook_ap_11, 'ending_orderbook_as_11':ending_orderbook_as_11,
                                                          'ending_orderbook_ap_12':ending_orderbook_ap_12, 'ending_orderbook_as_12':ending_orderbook_as_12,
                                                         'ending_orderbook_ap_13':ending_orderbook_ap_13, 'ending_orderbook_as_13':ending_orderbook_as_13,
                                                          'ending_orderbook_ap_14':ending_orderbook_ap_14, 'ending_orderbook_as_14':ending_orderbook_as_14,
                                                         'ending_orderbook_bp_0':ending_orderbook_bp_0, 'ending_orderbook_bs_0':ending_orderbook_bs_0,
                                                         'ending_orderbook_bp_1':ending_orderbook_bp_1, 'ending_orderbook_bs_1':ending_orderbook_bs_1,
                                                         'ending_orderbook_bp_2':ending_orderbook_bp_2, 'ending_orderbook_bs_2':ending_orderbook_bs_2,
                                                         'ending_orderbook_bp_3':ending_orderbook_bp_3, 'ending_orderbook_bs_3':ending_orderbook_bs_3,
                                                         'ending_orderbook_bp_4':ending_orderbook_bp_4, 'ending_orderbook_bs_4':ending_orderbook_bs_4,
                                                         'ending_orderbook_bp_5':ending_orderbook_bp_5, 'ending_orderbook_bs_5':ending_orderbook_bs_5,
                                                         'ending_orderbook_bp_6':ending_orderbook_bp_6, 'ending_orderbook_bs_6':ending_orderbook_bs_6,
                                                         'ending_orderbook_bp_7':ending_orderbook_bp_7, 'ending_orderbook_bs_7':ending_orderbook_bs_7,
                                                         'ending_orderbook_bp_8':ending_orderbook_bp_8, 'ending_orderbook_bs_8':ending_orderbook_bs_8,
                                                         'ending_orderbook_bp_9':ending_orderbook_bp_9, 'ending_orderbook_bs_9':ending_orderbook_bs_9,
                                                         'ending_orderbook_bp_10':ending_orderbook_bp_10, 'ending_orderbook_bs_10':ending_orderbook_bs_10,
                                                         'ending_orderbook_bp_11':ending_orderbook_bp_11, 'ending_orderbook_bs_11':ending_orderbook_bs_11,
                                                         'ending_orderbook_bp_12':ending_orderbook_bp_12, 'ending_orderbook_bs_12':ending_orderbook_bs_12,
                                                         'ending_orderbook_bp_13':ending_orderbook_bp_13, 'ending_orderbook_bs_13':ending_orderbook_bs_13,
                                                         'ending_orderbook_bp_14':ending_orderbook_bp_14, 'ending_orderbook_bs_14':ending_orderbook_bs_14})

                        # print(data_to_insert)
                        my_data = pd.concat([my_data, data_to_insert.to_frame().T], ignore_index=True)

                        
                        ########### 이 자리에 트레이딩 머신을 붙여야 한다 #############
                        # 10초씩 raw 데이터가 정리된 상황. my_data를 분석하여 사고 파는 결정을 내린다.
                        
                        
                        
                        
                        
                        
                        
                        
                        
                        # my_data가 쌓이면서 너무 길어지는 것을 방지하기 (old data는 삭제한다):
                        while (my_data.shape[0] > 100):
                            print("CHECKER#8")

                            my_data = my_data.drop([0])

                        print(my_data)
                        
                print("CHECKER#9")
                        
            except pd.errors.EmptyDataError:
                print('********************')
                time.sleep(0.1)

                #continue
            except pd.errors.ParserError:
                print('^^^^^^^^^^^^^^^^^^^^')
                time.sleep(0.1)    






    #     now_after = datetime.datetime.now()

    #     diff = now_after - now

            # Wait until the next unit second
    #         time.sleep(10-1 - now.microsecond / 1000000.0)

In [3]:
async def run_mani():
    
    lock = asyncio.Lock()
    await asyncio.wait([
        asyncio.create_task(data_manipulation(lock)),
    ])
    

In [4]:
if __name__ == "__main__":
    asyncio.run(run_mani())

CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#1
inside:
now: 2023-05-08 20:38:50.000645
CHECKER#3
CHECKER#3
CHECKER#4
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#6
CHECKER#7
                datetime_10s beginning_orderbook_midprice beginning_price  \
0 2023-05-08 20:38:40.036112                   37476000.0      37492000.0   

  highest_price lowest_price ending_price ending_orderbook_midprice  \
0    37492000.0   37460000.0   37460000.0                37476000.0   

  trading_volume volume_power trade_volume_bid  ... ending_orderbook_bp_10  \
0       1.445593    -1.195467         0.125063  ...               0.999199   

  ending_orderbook_bs_10 ending_orderbook_bp_11 end

CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#1
inside:
now: 2023-05-08 20:39:20.111585
CHECKER#3
CHECKER#3
CHECKER#4
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3

CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CH

CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECK

********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECK

CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#1
inside:
now: 2023-05-08 20:40:30.000094
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#4
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#3
CHECKER#6
CHECKER#7
                 datetime_10s beginning_orderbook_midprice beginning_price  \
0  2023-05-08 20:38:40.036112                   37476000.0      37492000.0   
1  2023-05-08 20:38:50.838284                   37476000.0      37460000.0   
2  2023-05-08 20:39:00.186765                   37470500.0      37460000.0   
3  2023-05-08 20:39:10.262955                   37470000.0      37460000.0   
4  2023-05-08 20:39:21.246611                   37459500.0      374

CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#

CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CH

CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
********************
CHECKER#0
********************
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2
CHECKER#0
CHECKER#0_1
CHECKER#0_2


KeyboardInterrupt: 

In [45]:
df.shape

(1005, 98)

In [ ]:
while True:
    try:
        df = pd.read_csv('temp3.csv')
        print(df.shape[0])
        
        
    except pd.errors.EmptyDataError:
        print('********************')
        
        #continue
    except pd.errors.ParserError:
        print('^^^^^^^^^^^^^^^^^^^^')
        
        #continue

In [ ]:
while True:
    try:
        df = pd.read_csv("temp3.csv")

        df['mytime'] = pd.to_datetime(df['mytime'],errors='raise')
        print("row length:", df.shape[0])
        print("difference:",(df['mytime'].iloc[-1] - df['mytime'].iloc[0]).total_seconds())
        print("")
        
        time.sleep(0.2)
    except pd.errors.EmptyDataError:
        #print('********************')
        time.sleep(0.2)
        # continue
    except pd.errors.ParserError:
        # print('^^^^^^^^^^^^^^^^^^^^')
        time.sleep(0.2)
        # continue

In [ ]:
df